# **1. Use python "requests" package to get the file from here**:
https://raw.githubusercontent.com/victorbrub/data-engineeringclass/refs/heads/main/pre-post_processing/exercise.csv

In [3]:
import requests
import pandas as pd
import numpy as np
import re

#Descargar el archivo csv
url = "https://raw.githubusercontent.com/victorbrub/data-engineering-class/refs/heads/main/pre-post_processing/exercise.csv"
raw_file = "raw.csv"

r = requests.get(url)
with open(raw_file, "wb") as f:
    f.write(r.content)
print("Archivo descargado.")

Archivo descargado.


# **2. Check the file data to fast check. Read it with Pandas.**

## a) How many rows do we have?

In [4]:
#Leer archivo con pandas
df = pd.read_csv((raw_file), on_bad_lines='skip')
print("Filas iniciales:", len(df))
df.head()

Filas iniciales: 189


,OrderID,CustomerName,Email,Phone,Country,OrderDate,Quantity,Price,CustomerAge,OrderStatus
0,1001,John Doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
1,1002,jane smith,JANE@GMAIL.COM,555-2345,united states,2023-01-16,2,49.99,35,Completed
2,1003,john doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
3,1004,Bob Johnson,bob.j@email.com,555-3456,US,2023-01-17,-1,99.99,45,Pending
4,1006,Charlie Davis,NaN,555-5678,UK,2023-02-19,3,79.99,unknown,Completed


In [5]:
df.shape

(189, 10)

The dataset has **dimensions** of **189 rows x 10 columns**

## b) Is there any sensible information?

From df.head(), it can be observed that the dataset contains several types of sensitive information:

Direct Personal Information (PII):
- **CustomerName** --> real names and surnames
- **Email** --> personal email addresses
- **Phone** --> phone numbers
<br>

Indirect personal data (becomes personal when linked):

- **OrderID** --> unique identifiers linked to an individual
- **OrderDate** --> dates associated with customer actions
- **Price** --> economic behavior
- **Quantity** --> consumption patterns
- **CustomerAge** --> non-sensitive personal characteristic
<br>

Inspecting the dataset using df.head() reveals that it contains personally identifiable information (PII) such as "CustomerName", "Email", and "Phone".

Other fields, such as "OrderID", "OrderDate", "Price", "Quantity", and "CustomerAge", are not personal data per se, but become personal information when directly linked to an individual. Therefore, the dataset requires appropriate handling from a privacy and GDPR compliance perspective.

## c) What kind of problems can we have regarding the nature of this data?

In [6]:
df.describe()

,OrderID,Quantity,Price
count,189.000000,189.000000,168.000000
mean,1095.978836,214.333333,2673.269762
std,54.739903,2909.376650,33942.620220
min,1001.000000,-5.000000,14.990000
25%,1049.000000,1.000000,29.990000
50%,1096.000000,3.000000,44.990000
75%,1143.000000,5.000000,74.990000
max,1190.000000,40000.000000,440000.990000


In [7]:
df.isnull().sum()

,0
OrderID,0
CustomerName,0
Email,24
Phone,5
Country,0
OrderDate,0
Quantity,0
Price,21
CustomerAge,2
OrderStatus,0


In [8]:
# Inspección automática de posibles problemas

# 1. Valores únicos en Country (inconsistencia en formatos)
print("Valores únicos en 'Country':\n", df['Country'].unique(), "\n")


# 2. Edades no numéricas
non_numeric_ages = df[~df['CustomerAge'].astype(str).str.isnumeric()]
print("Edades no numéricas en 'CustomerAge':\n", non_numeric_ages.head(), "\n")


# 3. Edades fuera de rango (0–122)
numeric_ages = df[df['CustomerAge'].astype(str).str.isnumeric()]
ages_out_of_range = numeric_ages[(numeric_ages['CustomerAge'].astype(int) < 0) |
                                 (numeric_ages['CustomerAge'].astype(int) > 122)]
print("Edades fuera de rango:\n", ages_out_of_range, "\n")


# 4. Cantidades negativas
print("Cantidad con valores negativos:\n", df[df['Quantity'] < 0], "\n")

Valores únicos en 'Country':
 ['USA' 'united states' 'US' 'UK' 'GB' 'usa' 'United Kingdom' 'us'
 'United States' 'Canada' 'canada' 'uk'] 

Edades no numéricas en 'CustomerAge':
     OrderID   CustomerName              Email     Phone Country   OrderDate  \
4      1006  Charlie Davis                NaN  555-5678      UK  2023-02-19   
5      1007   David Miller    david@email.com  555-6789      GB  2023-02-20   
18     1020    Peter Clark                NaN  555-7777     USA  2023-03-08   
19     1021        Antonio  antonio@email.com  555-7777     USA  2023-03-08   
24     1026      Tom Baker    tom.baker@email  555-1212      UK  2023-03-12   

    Quantity      Price CustomerAge OrderStatus  
4          3      79.99     unknown   Completed  
5          1     199.99          -5  Processing  
18         4      44.99         NaN   Completed  
19     40000  440000.99         NaN   Completed  
24         8      19.99     unknown   Completed   

Edades fuera de rango:
      OrderID       Cu

After analyzing the dataset, several data quality issues were observed.

First, there are significant **completeness problems**, as columns such as "Email", "Price", and "CustomerAge" contain null values in a significant number of records.

**Validity and accuracy problems** also appear, particularly noticeable in CustomerAge, where non-numeric ages (“unknown”) and impossible values such as negative ages or ages greater than 122 are found. Negative values are identified in the Quantity column, which is illogical in the context of a purchase order.

Regarding **consistency**, the Country column shows multiple variations for the same country ("USA", "US", "usa", "United States"), which hinders aggregation and future analysis. Furthermore, there are extremely high or inconsistent prices, suggesting data entry errors or a lack of validation at the source.

Finally, although not yet verified, there could be **uniqueness** issues if duplicate OrderIDs exist. Overall, the dataset exhibits typical errors of uncontrolled data that directly affect the main quality dimensions, and therefore requires a thorough cleaning process before any analysis.

# **3. Clean it**

## a) Define the rules we need to clean the data.

Next, we define a set of rules based on the problems detected during the preliminary analysis:

1. Format normalization: Standardize column names to lowercase and remove spaces.

2. Cleaning the CustomerAge column:

   - Convert numeric values correctly.

   - Replace non-numeric values ("unknown") with NaN.

   - Set ages outside the 0-122 range as invalid.

3. Email validation:

   - A valid email address must contain "@" and a domain.

   - Invalid email addresses are replaced with NaN.

4. Correction of inconsistencies in Country:

   - Standardize variants such as "US", "usa", "United States" --> "USA".

5. Correction of quantities:

   - Negative values in Quantity are considered invalid and are replaced with NaN.

6. Detection and removal of duplicates:

   - The dataset must have only one record per OrderID.

7. Date Conversion:

   - Convert OrderDate to datetime format. Invalid values --> NaN.

8. Removal or Correction of Invalid Prices:

   - Negative or excessively high prices are considered invalid --> NaN.

<br>

These rules allow you to prepare a consistent dataset suitable for subsequent quality tests.

## b) Implement a Python Script that cleans the data and analyzes the clean process.

In [9]:
# Copy of the raw dataset
clean = df.copy()
print("Rows before cleaning:", len(df))



# 1. Normalize column names
clean.columns = clean.columns.str.strip().str.lower()


# 2. Clean CustomerAge
clean['customerage'] = pd.to_numeric(clean['customerage'], errors='coerce')

# Valid range 0–122
invalid_age_mask = (clean['customerage'] < 0) | (clean['customerage'] > 122)
invalid_ages = invalid_age_mask.sum()

clean.loc[invalid_age_mask, 'customerage'] = np.nan


# 3. Email validation
email_pattern = r".+@.+\..+"

invalid_email_mask = ~clean['email'].astype(str).str.match(email_pattern, na=False)
invalid_emails = invalid_email_mask.sum()

clean.loc[invalid_email_mask, 'email'] = np.nan


# 4. Standardize Country
country_map = {
    'usa': 'USA', 'us': 'USA', 'United States': 'USA', 'united states': 'USA',
    'uk': 'UK', 'United Kingdom': 'UK', 'gb': 'UK'
}

clean['country'] = clean['country'].astype(str).str.strip().map(lambda x: country_map.get(x, x))


# 5. Fix Quantity (negative = invalid)
invalid_qty_mask = clean['quantity'] < 0
invalid_qty = invalid_qty_mask.sum()

clean.loc[invalid_qty_mask, 'quantity'] = np.nan


# 6. Price validation (remove negative or unrealistic values)
invalid_price_mask = (clean['price'] < 0) | (clean['price'] > 5000)
invalid_prices = invalid_price_mask.sum()

clean.loc[invalid_price_mask, 'price'] = np.nan


# 7. Fix OrderDate: handle mixed formats ("/" and "-")
clean['orderdate'] = clean['orderdate'].astype(str).str.replace("/", "-", regex=False)
clean['orderdate'] = pd.to_datetime(clean['orderdate'], errors='coerce')

invalid_dates = clean['orderdate'].isna().sum() - df['OrderDate'].isna().sum()


# 8. Remove duplicated OrderID
duplicates = df['OrderID'].duplicated().sum()
clean = clean.drop_duplicates(subset='orderid')


# Summary of cleaning impact
print("\n=== Cleaning Summary ===")
print("Invalid ages replaced with NaN:", invalid_ages)
print("Invalid emails removed:", invalid_emails)
print("Invalid quantities:", invalid_qty)
print("Invalid prices:", invalid_prices)
print("Invalid dates introduced:", invalid_dates)
print("Duplicate OrderIDs removed:", duplicates)
print("\nRows after cleaning:", len(clean))
print("\nNulls before cleaning:\n", df.isnull().sum())
print("\nNulls after cleaning:\n", clean.isnull().sum())

# Show cleaned DF
clean.head()

Rows before cleaning: 189

=== Cleaning Summary ===
Invalid ages replaced with NaN: 68
Invalid emails removed: 47
Invalid quantities: 26
Invalid prices: 1
Invalid dates introduced: 36
Duplicate OrderIDs removed: 0

Rows after cleaning: 189

Nulls before cleaning:
 OrderID          0
CustomerName     0
Email           24
Phone            5
Country          0
OrderDate        0
Quantity         0
Price           21
CustomerAge      2
OrderStatus      0
dtype: int64

Nulls after cleaning:
 orderid          0
customername     0
email           47
phone            5
country          0
orderdate       36
quantity        26
price           22
customerage     98
orderstatus      0
dtype: int64


,orderid,customername,email,phone,country,orderdate,quantity,price,customerage,orderstatus
0,1001,John Doe,john@email.com,555-1234,USA,2023-01-15,5.0,19.99,28.0,Completed
1,1002,jane smith,JANE@GMAIL.COM,555-2345,USA,2023-01-16,2.0,49.99,35.0,Completed
2,1003,john doe,john@email.com,555-1234,USA,2023-01-15,5.0,19.99,28.0,Completed
3,1004,Bob Johnson,bob.j@email.com,555-3456,US,2023-01-17,NaN,99.99,45.0,Pending
4,1006,Charlie Davis,NaN,555-5678,UK,2023-02-19,3.0,79.99,NaN,Completed


## c) Create a new file with cleaned data.

In [10]:
clean.to_csv("cleaned.csv", index=False)
print("cleaned.csv successfully saved.")

cleaned.csv successfully saved.


## d) Create a test on the raw data and on the cleaned data, for each one of the dimensions. The output should be a percentage: rows that meet the test requirement / total rows. Compare the results of each file.

In [13]:
# 1. Define Test Functions

def test_accuracy(df):
    #Age must be between 0 and 122.
    if 'customerage' not in df.columns:
        return 0

    ages = pd.to_numeric(df['customerage'], errors='coerce')
    mask = ages.between(0, 122, inclusive="both")
    return mask.mean()

def test_completeness(df):
    #No missing values in important columns.
    important_cols = ['email', 'price', 'quantity', 'customerage']
    available_cols = [col for col in important_cols if col in df.columns]
    return df[available_cols].notna().all(axis=1).mean()


def test_consistency(df):
    #Country must be standardized.
    valid_countries = ['USA', 'UK', 'Canada']
    return df['country'].isin(valid_countries).mean()


def test_validity(df):
    #Email must follow a valid format.
    pattern = r".+@.+\..+"
    return df['email'].astype(str).str.match(pattern, na=False).mean()


def test_uniqueness(df):
    #OrderID must be unique.
    return (df['orderid'].duplicated() == False).mean()


def test_timeliness(df):
    #A row is timely if it has a valid date and belongs to the year 2023.
    if 'orderdate' not in df.columns:
        return 0

    dates = pd.to_datetime(df['orderdate'], errors='coerce', infer_datetime_format=True)
    mask = dates.notna() & (dates.dt.year == 2023)
    return mask.mean()


# 2. Run Tests on raw and clean (raw must have lowercase columns to match clean dataset)
df_raw = df.copy()
df_raw.columns = df_raw.columns.str.lower()

tests_raw = {
    "Accuracy": test_accuracy(df_raw),
    "Completeness": test_completeness(df_raw),
    "Consistency": test_consistency(df_raw),
    "Validity": test_validity(df_raw),
    "Uniqueness": test_uniqueness(df_raw),
    "Timeliness": test_timeliness(df_raw)
}

tests_clean = {
    "Accuracy": test_accuracy(clean),
    "Completeness": test_completeness(clean),
    "Consistency": test_consistency(clean),
    "Validity": test_validity(clean),
    "Uniqueness": test_uniqueness(clean),
    "Timeliness": test_timeliness(clean)
}



# 3. Display Comparison Table
results = pd.DataFrame([tests_raw, tests_clean], index=["RAW", "CLEAN"])
results.style.format("{:.2%}")

/tmp/ipython-input-2087762758.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(df['orderdate'], errors='coerce', infer_datetime_format=True)
/tmp/ipython-input-2087762758.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(df['orderdate'], errors='coerce', infer_datetime_format=True)


,Accuracy,Completeness,Consistency,Validity,Uniqueness,Timeliness
RAW,48.15%,77.78%,50.26%,75.13%,100.00%,62.96%
CLEAN,48.15%,28.04%,83.07%,75.13%,100.00%,80.95%


The data quality evaluation clearly shows how the cleaning process improved several key dimensions of the dataset.

After cleaning, **consistency** and **timeliness** show substantial improvements. This is mainly because country names were standardized and date formats were corrected, which increases the semantic reliability of the data. **Accuracy** and **validity** remain unchanged, as the cleaning process replaces incorrect values with NaN but does not artificially generate new valid ones.

On the other hand, **completeness** decreases significantly in the cleaned dataset. This effect is expected: enforcing strict validation rules inevitably removes incorrect or inconsistent values, leading to more missing data. While the raw dataset appears more complete, it contains many values that are inaccurate, inconsistent, or invalid—meaning that the apparent completeness is misleading.

Finally, **uniqueness** stays at 100% in both versions, confirming that the dataset contains no duplicated OrderID values.

<br>

Overall, the cleaned dataset offers higher-quality, more trustworthy information, even though it contains more missing fields. This reflects a fundamental principle of data engineering: improving data quality often requires sacrificing completeness in order to ensure accuracy, consistency, and reliability.

##